# Try using the SSSC data 

In [1]:
import pandas
from pandas.testing import assert_frame_equal
from lsst.rsp import get_tap_service, retrieve_query
pandas.set_option('display.max_rows', 20)

In [2]:
# Get the default tap service using the old interface
# note that it gives a deprecation warning - we will not use this. 
service = get_tap_service()

/tmp/ipykernel_2836/1975941569.py:3: DeprecationWarning: get_tap_service() is deprecated, use get_tap_service("tap")
  service = get_tap_service()


In [3]:
service = get_tap_service('tap')
assert service is not None
assert service.baseurl == "https://data-int.lsst.cloud/api/tap"

In [4]:
query = """
SELECT * FROM tap_schema.schemas
"""

In [5]:
results = service.search(query)
print(type(results))

<class 'pyvo.dal.tap.TAPResults'>


In [6]:
results = service.search(query).to_table()
results

description,schema_index,schema_name,utype
str512,int32,str64,str512
Data Preview 0.1 includes five tables based on the DESC's Data Challenge 2 simulation of 300 square degrees of the wide-fast-deep LSST survey region after 5 years. All tables contain objects detected in coadded images.,1,dp01_dc2_catalogs,
"Data Preview 0.2 contains the image and catalog products of the Rubin Science Pipelines v23 processing of the DESC Data Challenge 2 simulation, which covered 300 square degrees of the wide-fast-deep LSST survey region over 5 years.",0,dp02_dc2_catalogs,
ObsCore v1.1 attributes in ObsTAP realization,2,ivoa,
A TAP-standard-mandated schema to describe tablesets in a TAP 1.1 service,100000,tap_schema,
UWS Metadata,120000,uws,


See that the standard default tap service provides access to the DP0.1/2 tables

In [7]:
# Now we will get an sso specific tap service 
sso_tap = get_tap_service("ssotap")
assert service is not None
print(sso_tap.baseurl)
assert sso_tap.baseurl == "https://data-int.lsst.cloud/api/ssotap"

https://data-int.lsst.cloud/api/ssotap


In [8]:
results = sso_tap.search(query)

In [9]:
results = sso_tap.search(query).to_table()
results

description,schema_index,schema_name,utype
str512,int32,str64,str512
Data Preview 0.3 contains the catalog products of a Solar System Science Collaboration simulation of the results of SSO analysis of the wide-fast-deep data from the LSST ten-year dataset.,0,dp03_catalogs,
A TAP-standard-mandated schema to describe tablesets in a TAP 1.1 service,100000,tap_schema,
UWS Metadata,120000,uws,


In [10]:
schema_names = results['schema_name']
for name in schema_names:
    if name.find('dp03') > -1:
        dp03_schema_name = name
        break
print("DP0.3 schema is " + dp03_schema_name)

DP0.3 schema is dp03_catalogs


In [11]:
query = """
SELECT * FROM tap_schema.tables
WHERE tap_schema.tables.schema_name = 
'dp03_catalogs' 
ORDER by table_index ASC
"""

In [12]:
results = sso_tap.search(query).to_table()
results

description,schema_name,table_index,table_name,table_type,utype
str512,str512,int32,str64,str8,str512
The orbit catalog produced by the Minor Planet Center. Ingested daily. O(10M) rows by survey end. The columns are described at https://minorplanetcenter.net//iau/info/MPOrbitFormat.html,dp03_catalogs,0,dp03_catalogs.MPCORB,table,
"LSST-computed per-object quantities. 1:1 relationship with MPCORB. Recomputed daily, upon MPCORB ingestion.",dp03_catalogs,0,dp03_catalogs.SSObject,table,
"LSST-computed per-source quantities. 1:1 relationship with DIASource. Recomputed daily, upon MPCORB ingestion.",dp03_catalogs,0,dp03_catalogs.SSSource,table,
Table to store 'difference image sources'; - sources detected at SNR >=5 on difference images.,dp03_catalogs,0,dp03_catalogs.DiaSource,table,
